# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

哪些學生需要介入輔導？這個問題是監督式學習裡面的分類或回歸？

**Answer: ** This is a classification problem. Given a set of students' data, we have to identify each whether he/she needs early intervention or not. This is obviously a binary classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
student_data.shape

(395, 31)

In [3]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = sum(student_data['passed']=='yes')

# TODO: Calculate failing students
n_failed = sum(student_data['passed']=='no')

# TODO: Calculate graduation rate
grad_rate = (n_passed*100.0)/(n_passed+n_failed)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
<bound method DataFrame.head of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
5       GP   M   16       U     LE3       T     4     3  services     other   
6       GP   M   16       U     LE3       T     

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

把yes/no轉換成0/1

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

類別變數中，多重類別的處理方式

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

pandas中有get_dummies()函數可以處理

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [13]:
print X_all.head

<bound method DataFrame.head of      school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  \
0            1          0      1      0   18          0          1   
1            1          0      1      0   17          0          1   
2            1          0      1      0   15          0          1   
3            1          0      1      0   15          0          1   
4            1          0      1      0   16          0          1   
5            1          0      0      1   16          0          1   
6            1          0      0      1   16          0          1   
7            1          0      1      0   17          0          1   
8            1          0      0      1   15          0          1   
9            1          0      0      1   15          0          1   
10           1          0      1      0   15          0          1   
11           1          0      1      0   15          0          1   
12           1          0      0      1   15          0   

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.
  
用70%,25%將data分成training,testing

In [17]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.24, random_state=1)



# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

選3種監督式學習的方法，評估其優缺點

計算F1

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **  The three models I chose here are the Decision trees, SVMs, and Gaussian Naive Bayes classifiers. A decision tree is a flowchart-like structure in which each internal node represents a "test" on an attribute (e.g. whether a coin flip comes up heads or tails), each branch represents the outcome of the test and each leaf node represents a class label (decision taken after computing all attributes). The path from root to leaf represents classification rules. The advantages of decision tree are that it is easy to understand, does not require large amount of data, and can handle both numerical and categorical data. The major weakness of decision tree is that it is easy to overfit data. The learning algorithm may generate too many rules in order to classify a noise to a desired label. The reason why I chose Decision tree here is that the given data is heterogeneous (that is, features include not only numerical data, but also categorical data). 

SVM is a power model which can perform nonlinear separation in current feature space. Many applications (especially binary classification problems) adopt SVM as the classification tool. For example, pedestrian detection in computer vision field. The advantages of SVMs are that they are good at dealing with higher dimensional problems, besides, they only consider a small part of data (called the support vectors) to determine the decision hyperplanes. That is, the model doesn't need lots of training data to find a hyperplane. In general, SVM usually doesn't overfit the data. The disadvantage of SVM is that the performance is highly rely on the kernel function. That is, we may not find a good hyperplane if we use a wrong kernel function. SVMs are chosen here mainly because the given data set has many futures, that is, the data set is a higher dimensional one, which may be well manipulated by a SVM.

Gaussian Naive Bayes classifier is a probabilistic based method. A famous application is the spam filter (based on counting word frequencies). The advantage of Naive Bayes classifier is that it is very efficient and we can infer the result given only a part of features. More importantly, Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The disadvantage is that the naive Bayes classifier assumes that features are conditionally independent. This assumption is not suitable in some applications such as digit recognition. Another disadvantage is that if the occurrence of a certain feature is zero in one class, the posterior probability will be zero. Usually we apply a smoother (e.g. Laplace smoother) to remedy this problem. The reasons why I chose Naive Bayes classifiers here are: (1) it is very fast; (2) it is very simple (from the perspective of implementation). In the near future, I would like to implement some machine learning algorithms in a resource-restricted system (e.g., embedded systems for IoT applications). That system has no Scikit-learn package; and everything need to be implemented by myself. I want to experiment whether a Naive Bayes classifier is good enough for dealing with such a classification problem.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    total_time= end-start
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)
    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    total_time= end-start
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes'),total_time


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1_train,time_train=predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {:.4f}.".format(f1_train)
    
    f1_test,time_test=predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {:.4f}.".format(f1_test)
    return f1_train,time_train,f1_test,time_test

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [9]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A (Decision tree)
from sklearn.tree import DecisionTreeClassifier
# from sklearn import model_B (SVM)
from sklearn.svm import SVC
# from skearln import model_C (Neural networks)
from sklearn.naive_bayes import GaussianNB

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=1) 
clf_B = SVC(random_state=1)
clf_C = GaussianNB()

# TODO: Set up the training set sizes
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

f1_A_100_train,time_A_100_train,f1_A_100_test,time_A_100_test=train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
f1_A_200_train,time_A_200_train,f1_A_200_test,time_A_200_test=train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
f1_A_300_train,time_A_300_train,f1_A_300_test,time_A_300_test=train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

f1_B_100_train,time_B_100_train,f1_B_100_test,time_B_100_test=train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
f1_B_200_train,time_B_200_train,f1_B_200_test,time_B_200_test=train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
f1_B_300_train,time_B_300_train,f1_B_300_test,time_B_300_test=train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

f1_C_100_train,time_C_100_train,f1_C_100_test,time_C_100_test=train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
f1_C_200_train,time_C_200_train,f1_C_200_test,time_C_200_test=train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
f1_C_300_train,time_C_300_train,f1_C_300_test,time_C_300_test=train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0050 seconds
Made predictions in 0.0025 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6829.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0019 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7023.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0021 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6984.
Training a SVC using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.8333.
Training a SVC using a training set size of 200. . 

### Print the results in Markdown

In [10]:
print '** Classifer 1 - Decision Tree**'
print
print '| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |'
print '| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |'
print "| 100 | ","{:.4f}".format(time_A_100_train), "|",  "{:.4f}".format(time_A_100_test), "|",
print "{:.4f}".format(f1_A_100_train),"|","{:.4f}".format(f1_A_100_test),"|"
print "| 200 | ","{:.4f}".format(time_A_200_train), "|",  "{:.4f}".format(time_A_200_test), "|",
print "{:.4f}".format(f1_A_200_train),"|","{:.4f}".format(f1_A_200_test),"|"
print "| 300 | ","{:.4f}".format(time_A_300_train), "|",  "{:.4f}".format(time_A_300_test), "|",
print "{:.4f}".format(f1_A_300_train),"|","{:.4f}".format(f1_A_300_test),"|"

print '** Classifer 2 - SVM**'
print
print '| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |'
print '| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |'
print "| 100 | ","{:.4f}".format(time_B_100_train), "|",  "{:.4f}".format(time_B_100_test), "|",
print "{:.4f}".format(f1_B_100_train),"|","{:.4f}".format(f1_B_100_test),"|"
print "| 200 | ","{:.4f}".format(time_B_200_train), "|",  "{:.4f}".format(time_B_200_test), "|",
print "{:.4f}".format(f1_B_200_train),"|","{:.4f}".format(f1_B_200_test),"|"
print "| 300 | ","{:.4f}".format(time_B_300_train), "|",  "{:.4f}".format(time_B_300_test), "|",
print "{:.4f}".format(f1_B_300_train),"|","{:.4f}".format(f1_B_300_test),"|"

print '** Classifer 3 - Gaussian Naive Bayes**'
print
print '| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |'
print '| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |'
print "| 100 | ","{:.4f}".format(time_C_100_train), "|",  "{:.4f}".format(time_C_100_test), "|",
print "{:.4f}".format(f1_C_100_train),"|","{:.4f}".format(f1_C_100_test),"|"
print "| 200 | ","{:.4f}".format(time_C_200_train), "|",  "{:.4f}".format(time_C_200_test), "|",
print "{:.4f}".format(f1_C_200_train),"|","{:.4f}".format(f1_C_200_test),"|"
print "| 300 | ","{:.4f}".format(time_C_300_train), "|",  "{:.4f}".format(time_C_300_test), "|",
print "{:.4f}".format(f1_C_300_train),"|","{:.4f}".format(f1_C_300_test),"|"


** Classifer 1 - Decision Tree**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |  0.0025 | 0.0010 | 1.0000 | 0.6829 |
| 200 |  0.0006 | 0.0003 | 1.0000 | 0.7023 |
| 300 |  0.0004 | 0.0002 | 1.0000 | 0.6984 |
** Classifer 2 - SVM**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |  0.0011 | 0.0007 | 0.8591 | 0.8333 |
| 200 |  0.0021 | 0.0010 | 0.8581 | 0.8408 |
| 300 |  0.0042 | 0.0016 | 0.8584 | 0.8462 |
** Classifer 3 - Gaussian Naive Bayes**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
|

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |  0.0000 | 0.0000 | 1.0000 | 0.6667 |
| 200 |  0.0010 | 0.0010 | 1.0000 | 0.7023 |
| 300 |  0.0010 | 0.0010 | 1.0000 | 0.6984 |
** Classifer 2 - SVM**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |  0.0020 | 0.0020 | 0.8591 | 0.8333 |
| 200 |  0.0080 | 0.0020 | 0.8581 | 0.8408 |
| 300 |  0.0110 | 0.0020 | 0.8584 | 0.8462 |
** Classifer 3 - Gaussian Naive Bayes**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |  0.0000 | 0.0010 | 0.8346 | 0.7402 |
| 200 |  0.0000 | 0.0010 | 0.7879 | 0.6446 |
| 300 |  0.0010 | 0.0010 | 0.7921 | 0.6720 |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **  I choose the SVM model as the best model. The Decision Tree (DT) model runs very fast and have perfect F1 score in training data. But the f1 score of testing set is low. That is, the trained model overfits the data. The performance of Naive Bayes classifier stands between DT and SVM. The Naive Bayes classifier does not perform well with testing data. It might because the distribution of the small training dataset is not representative. On the other hand, SVM has a balanced f1 score among three models. Although it takes some training time but usually we don't need to train many times if we have a robust model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** In this project, we want to know whether a student will pass or not. In that case, students have to be classified into two classes: "pass" or "not pass". In the training phase, SVM tries to find a best separate line to separate students into correct classes. The term "best" is defined as maximizing the distance between the separate line and both classes. This line will separate the entire space into two regions. To make a prediction, we simply check which region a new data falls in. In some situations we cannot use a line to separate data correctly. However, by taking the distances from each point to the origin into account, we can find that these points can be separated perfectly. Methods to obtain new features like this is called the kernel trick. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer 

# TODO: Create the parameters list you wish to tune
parameters =  {'kernel':['linear','rbf','poly'],'C':[0.1,0.5,1,5,10,100],'gamma':[0,0.001,0.002,0.003,0.004,0.005,],'tol':[1e-2,1e-3,1e-4]} 

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer =  make_scorer(f1_score,greater_is_better = True,pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj =  GridSearchCV(clf,parameters,scoring=f1_scorer) 

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
f1_train,time_train=predict_labels(clf, X_train, y_train)
print "Tuned model has a training F1 score of {:.4f}.".format(f1_train)
f1_test,time_test=predict_labels(clf, X_test, y_test)
print "Tuned model has a testing F1 score of {:.4f}.".format(f1_test)

/Users/kimble/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Made predictions in 0.0036 seconds.
Tuned model has a training F1 score of 0.8609.
Made predictions in 0.0013 seconds.
Tuned model has a testing F1 score of 0.8533.


In [12]:
clf

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.004, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.01, verbose=False)

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final f1 score is 0.8584 and 0.8462 for training and testing set respectively. The result is identical to the untuned model. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.